# Evaluate CEMS coverage

1) What plant hours included in 923 are entirely missing from CEMS? 
2) What plant months have underreported CEMS generation compared to their 923-reported generation? 
3) Is the gap between 923 and CEMS consistent with 930 data? 

*Note: use processed CEMS data output by data_pipeline notebook.*

In [ ]:
import pandas as pd
import sqlalchemy as sa
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
year=2019

In [ ]:
# Read data output by data_pipeline
# Advantages over getting direct from PUDL: 
cems = pd.read_csv(f"../data/outputs/cems{year}.csv", parse_dates=["datetime_utc"])

In [ ]:
# specify the relative path to the sqllite database, and create an sqalchemy engine
pudl_db = 'sqlite:///../data/downloads/pudl/pudl_data/sqlite/pudl.sqlite'
pudl_engine = sa.create_engine(pudl_db)


# 1. What plant months in 923 are entirely missing from CEMS? 

In [ ]:
cems.head()

In [ ]:
# PUDL reports 923 monthly generation data across two tables, generation_eia923 and generation_fuel_eia923

# Load generation, filter for year. PUDL says that this only contains ~55% of reported generation 
# (https://catalystcoop-pudl.readthedocs.io/en/latest/data_sources/eia923.html)
# Table is large, so filter before load
gen_923 = pd.read_sql(f"SELECT * FROM generation_fuel_eia923 \
                           WHERE report_date >= '{year}-01-01' \
                           AND report_date <= '{year}-12-30'", pudl_engine)
gen_923.report_date = pd.to_datetime(gen_923.report_date)
gen_923.report_date = gen_923.report_date.dt.tz_localize("UTC")
gen_923.head()

In [ ]:
# Group 923 by plants because some plants report multiple fuel/prime mover combos in this table 
month_923 = gen_923.groupby(["plant_id_eia","report_date"]).sum()

In [ ]:
############################## IRRELEVANT: even annually reporting plants report monthly data. ###########################

# Some 923 data is reported monthly, some annually. To aggregate CEMS correctly, assign report_date to each CEMS 
# I *think* that PUDL converts dates to beginning-of-month: https://catalystcoop-pudl.readthedocs.io/en/latest/_modules/pudl/transform/eia923.html#_yearly_to_monthly_records
# TODO: this is slooooooooow, could maybe find a way to speed it up 
#       eg by assuming relationship between report date 

# cems["report_date"] = np.nan
# cems["report_date"] = pd.to_datetime(cems["report_date"])
# i=0
# for plant in cems.plant_id_eia.unique():
#     if i%100 == 0:
#         print(f"{i/len(cems.plant_id_eia.unique())}...",end="")
#     i += 1
#     if plant not in gen_923.plant_id_eia.index: # 35 CEMS plants missing from EIA-923
#         continue
#     # Collect 923 report dates for this plant 
#     this_plant_dates = gen_923[gen_923.plant_id_eia==plant].report_date
#     # At most we will have monthly resolution
#     for month in cems[cems.plant_id_eia == plant].datetime_utc.dt.month.unique():
        
#         # What is the latest 923 report date before the CEMS report date month? 
#         if len(this_plant_dates[this_plant_dates.dt.month <= month])==0:
#             print(f"month {month}, plant {plant} has no matching report date")
#             continue
#         target_date = max(this_plant_dates[this_plant_dates.dt.month <= month])

#         # Set all rows with this plant and month 
#         cems.loc[(cems.plant_id_eia == plant) & (cems.datetime_utc.dt.month==month),"923_report_date"] = target_date

In [ ]:
# Assign report date: first day of month in question
cems["report_date"] = pd.to_datetime({"month":cems.datetime_utc.dt.month, "year":cems.datetime_utc.dt.year,"day":1})\
    .dt.tz_localize("UTC")
month_cems = cems.groupby(["plant_id_eia","report_date"]).sum()

In [ ]:
# What plants are missing? 
print(f"# plants in CEMS not in 923 = {len(np.setdiff1d(cems.plant_id_eia, gen_923.plant_id_eia))}")
print(f"# plants in 923 not in CEMS = {len(np.setdiff1d(gen_923.plant_id_eia, cems.plant_id_eia))}")
print(f"\r\nTotal plants in 923 = {len(gen_923.plant_id_eia.unique())}")
print(f"Total plants in CEMS = {len(cems.plant_id_eia.unique())}")

In [ ]:
# Add CEMS generation numbers (allocated by month) to 932 numbers to compare month-by-month missing data
month_cems_merge = month_cems[["co2_mass_tons","heat_content_mmbtu","gross_generation_mwh","net_generation_mwh"]].rename(columns={\
    "co2_mass_tons":"co2_mass_tons_cems",\
    "heat_content_mmbtu":"heat_content_mmbtu_cems",\
    "gross_generation_mwh":"gross_generation_mwh_cems",\
    "net_generation_mwh":"net_generation_mwh_cems"})
month_923 = month_923.merge(month_cems_merge,how="left", \
    left_index=True, right_index=True)
month_923.head()

In [ ]:
# Add helper column to indicate where data is missing from CEMS 
month_923["missing_cems"] = month_923["heat_content_mmbtu_cems"].isna()
# 33 rows where fuel consumed in CEMS is 0 but not in 923
month_923.loc[(month_923["heat_content_mmbtu_cems"] == 0) & (month_923["fuel_consumed_mmbtu"] != 0),["missing_cems"]] = True

In [ ]:
# What is the generation of plants not in CEMS? 
not_cems_ids = np.setdiff1d(gen_923.plant_id_eia, cems.plant_id_eia)
shared_ids = np.intersect1d(cems.plant_id_eia, gen_923.plant_id_eia)

fig = go.Figure()
fig.add_trace(go.Histogram(x=month_923[~month_923.missing_cems]["fuel_consumed_mmbtu"], name="In CEMS and 923"))
fig.add_trace(go.Histogram(x=month_923[month_923.missing_cems]["fuel_consumed_mmbtu"],name="Not in CEMS"))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Make log 
fig.update_yaxes(type="log", title_text="# of plant months")
fig.update_xaxes(title_text="Fuel consumed in 923 reporting period (mmbtu)")
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
# What percent of hours are in CEMS?
print(f"{sum(month_923.missing_cems)/len(month_923.missing_cems)} plant-months in 923 are entirely missing in CEMS")

# 2. What plant-months have some CEMS data, but less than they report in 923? 

I don't know if this matters: eGRID documentation says that CEMS reports are taken over 923 (section 3.1.1.1 in 2020 technical guide)

In [ ]:
has_cems = month_923[~month_923.missing_cems].copy()
has_cems["fraction_reported_cems"] = (has_cems.fuel_consumed_mmbtu - has_cems.heat_content_mmbtu_cems)/has_cems.fuel_consumed_mmbtu
px.histogram(has_cems.loc[has_cems["fraction_reported_cems"].abs() < 3, "fraction_reported_cems"], log_y=False, range_x=(-1,1))

In [ ]:
# How does the # of plants not reporting cems change over time? 

counts = month_923.groupby("report_date").count()
px.line(counts, x=counts.index, y="missing_cems")

In [ ]:
# How does the production of plants not reporting cems change over time? 
no_cems = month_923[month_923.missing_cems].groupby("report_date").sum()

px.line(no_cems, x=no_cems.index, y="fuel_consumed_mmbtu")

In [ ]:
# Side note: the 923 fuel consumption data is not internally consistent. 

px.histogram((month_923.fuel_consumed_mmbtu - (month_923.fuel_consumed_units * month_923.fuel_mmbtu_per_unit))/month_923.fuel_consumed_mmbtu, log_y=True, \
    labels={"value":"mmBtu gap = (fuel_consumed_mmbtu - fuel_consumed_units*fuel_mmbtu_per_unit)/fuel_consumed_mmbtu"}, \
        title="Inconsistencies in EIA-923 fuel consumption metrics")

In [ ]:
month_923

# 3. Does availibility of CEMS data explain changes in CEMS relative to 930 over the year? 